In [11]:
# Skaičiuoja, kiek Zive įrašuose yra rpeaks, nesutampančių tarp tų, kurie rasti
# grynai automatiniu būdu ir tų, kurie yra pataisyti anotuojant įrašus ir saugomi json. 
# Rezultatai įrašomi į Excel (Windows atveju) ir csv failus, naudojau šiuos failus darbui.
# Nuskaitomi originalūs įrašai.  

import neurokit2 as nk
import pandas as pd
import numpy as np
from pathlib import Path
import os, sys, json
from zive_util_vu import zive_read_df_rpeaks, get_rec_file_name, split_SubjCode, get_recId, zive_read_file_1ch


def misalignments_org(rec_dir, file_name, print_flag=False):

       # I-a dalis: nuskaitome rpeaks iš json failo
    df_rpeaks = zive_read_df_rpeaks(rec_dir, file_name)
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    n_rpeaks = atr_sample.shape[0]
    if (print_flag):
        print('    json:',atr_sample[:10])

        # II-a dalis: suformuojame rpeaks su Neurokitu
    file_path = Path(rec_dir, file_name)
    signal_raw = zive_read_file_1ch(file_path)      
    _, rpeaks = nk.ecg_peaks(signal_raw, sampling_rate=200, correct_artifacts=False)
    rpeaks_from_signal = rpeaks['ECG_R_Peaks']
    if (print_flag):
        print('neurokit:',rpeaks_from_signal[:10])
  
    # Sulyginimas
    ab = np.setdiff1d(atr_sample, rpeaks_from_signal)
    ba = np.setdiff1d(rpeaks_from_signal,atr_sample)
    return(n_rpeaks, np.size(ab) + np.size(ba))


pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_2022_RUDUO_2'

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records')

# Variantas: visi duomenys
# Nuskaitomas įrašų sąrašas, suformuojamas atitinkamas dataframe df_list
file_path = Path(rec_dir, 'list.json')
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())
df_list = pd.json_normalize(data, record_path =['data'])
print("\nViso įrašų:", len(df_list), "\n")
# print(df_list.head())

df_ns = pd.DataFrame({'ind': pd.Series(dtype='int'),
                   'filename': pd.Series(dtype='str'),
                   'userId': pd.Series(dtype='str'),
                   'recId': pd.Series(dtype='str'),
                   'ns': pd.Series(dtype='int')
                   })

# Failas rezultatų išvedimui į failą
filename = "information_about_rpeaks_tst"

tot, ns_tot = 0, 0
dict_ns_tot = []

for ind, row in df_list.iterrows():
    userId = row['userId']
    recordingId = row['recordingId']
    file_name = row['file_name']
    # print(userId, file_name)

    n_rpeaks, ns = misalignments_org(rec_dir, file_name, print_flag=False)

    dict_ns = {'ind':ind, 'filename':str(file_name), 'userId':userId, 'recId':recordingId,'ns':ns}
    dict_ns_tot.append(dict_ns) 
    # if (ns != 0):
    pr_text = f"Zive įrašas: {ind} userId: {userId} recordingId: {recordingId} filename: {file_name} rpeaks: {n_rpeaks} nesutampančių rpeaks: {int(ns)}"
    print(pr_text)
    # n_rpeaks, ns = misalignments(rec_dir, SubjCode, print_flag=False)
    if (ns != 0):
        ns_tot +=1
    tot +=1

df_ns = pd.DataFrame(dict_ns_tot)

# Suformuojame sąraše naują kolonėlę su anotacijų kiekiu
df_ns['E'] = df_list['S'] + df_list['V'] + df_list['U']

print()
print(df_ns.head())

if (OS == 'Windows'): 
    # Windows - įrašome Excel lentelę su rezultatais
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html
    # filepath = Path(pred_path, pred_folder+'.xlsx')
    
    filepath = filename + '.xlsx'
    with pd.ExcelWriter(filepath) as writer:
        df_ns.to_excel(writer, sheet_name='rpeaks')
    print(f'\nFailų sąrašas ir rezultatai įrašyti į:  {filepath}')

# Taip pat įrašome dataframe csv pavidalu 
filepath = filename + '.csv'
df_ns.to_csv(filepath)
print(f'\nFailų sąrašas ir rezultatai įrašyti į:  {filepath}')    

pr_text = f"\nViso įrašų: {tot}  Įrašų su nesutampančiais rpeaks: {ns_tot} ({100*ns_tot/tot:.1f}%)"   
print(pr_text)



OS in my system :  win32

Viso įrašų: 2552 

Zive įrašas: 0 userId: 60a917b354352a3df86dc1f2 recordingId: 60a92aa454352a03116dc1f5 filename: 1621694.321 rpeaks: 6018 nesutampančių rpeaks: 0
Zive įrašas: 1 userId: 60e1d80f93b55b41529e9eaa recordingId: 60e1db5a93b55b6ab59e9ebe filename: 1625400.796 rpeaks: 818 nesutampančių rpeaks: 2
Zive įrašas: 2 userId: 60e1d80f93b55b41529e9eaa recordingId: 60e1db5a93b55b56919e9ec0 filename: 1625402.027 rpeaks: 765 nesutampančių rpeaks: 4
Zive įrašas: 3 userId: 60e1d80f93b55b41529e9eaa recordingId: 60e1db5a93b55be2659e9ec2 filename: 1625403.256 rpeaks: 817 nesutampančių rpeaks: 1
Zive įrašas: 4 userId: 60e1d80f93b55b41529e9eaa recordingId: 60e1db5a93b55b14359e9ec3 filename: 1625403.874 rpeaks: 1141 nesutampančių rpeaks: 6
Zive įrašas: 5 userId: 60e1d80f93b55b41529e9eaa recordingId: 60e1db5a93b55b2f5b9e9ec5 filename: 1625405.103 rpeaks: 1302 nesutampančių rpeaks: 10
Zive įrašas: 6 userId: 60e1d80f93b55b41529e9eaa recordingId: 60e1db5a93b55b6ad19e9ec6 f